## Imports

In [ ]:
import copy
import logging
from pathlib import Path
from typing import Dict
import math
import itertools
from ccmm.utils.utils import l2_norm_models
import hydra
import matplotlib
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import omegaconf
import seaborn as sns
import torch  # noqa
import wandb
from hydra.utils import instantiate
from matplotlib import tri
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
from omegaconf import DictConfig
from pytorch_lightning import LightningModule
from scipy.stats import qmc
from torch.utils.data import DataLoader
from tqdm import tqdm
from ccmm.matching.utils import perm_indices_to_perm_matrix
from ccmm.utils.utils import normalize_unit_norm, project_onto
from functools import partial

from nn_core.callbacks import NNTemplateCore
from nn_core.common import PROJECT_ROOT
from nn_core.common.utils import seed_index_everything
from nn_core.model_logging import NNLogger
from ccmm.utils.utils import fuse_batch_norm_into_conv
from torch.utils.data import DataLoader, Subset, SubsetRandomSampler

import ccmm  # noqa
from ccmm.matching.utils import (
    apply_permutation_to_statedict,
    get_all_symbols_combinations,
    plot_permutation_history_animation,
    restore_original_weights,
)
from ccmm.utils.utils import (
    linear_interpolate,
    load_model_from_info,
    map_model_seed_to_symbol,
    save_factored_permutations,
)

from ccmm.matching.utils import load_permutations

from ccmm.utils.utils import vector_to_state_dict, get_interpolated_loss_acc_curves
import pytorch_lightning

In [ ]:
logging.getLogger("lightning.pytorch").setLevel(logging.WARNING)
logging.getLogger("torch").setLevel(logging.WARNING)
logging.getLogger("pytorch_lightning.accelerators.cuda").setLevel(logging.WARNING)
pylogger = logging.getLogger(__name__)

In [ ]:
plt.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "serif",
    }
)
sns.set_context("talk")

cmap_name = "coolwarm_r"

from ccmm.utils.plot import Palette

palette = Palette(f"{PROJECT_ROOT}/misc/palette2.json")
palette

## Configuration

In [ ]:
%load_ext autoreload
%autoreload 2

import hydra
from hydra import initialize, compose
from typing import Dict, List

hydra.core.global_hydra.GlobalHydra.instance().clear()

initialize(version_base=None, config_path=str("../conf"), job_name="matching_n_models")
cfg = compose(config_name="matching_n_models", overrides=[])

In [ ]:
core_cfg = cfg  # NOQA
cfg = cfg.matching

seed_index_everything(cfg)

## Hyperparameters

In [ ]:
num_test_samples = 5000
num_train_samples = 5000

## Load dataset

In [ ]:
transform = instantiate(core_cfg.dataset.test.transform)

train_dataset = instantiate(core_cfg.dataset.train, transform=transform)
test_dataset = instantiate(core_cfg.dataset.test, transform=transform)

train_subset = Subset(train_dataset, list(range(num_train_samples)))
train_loader = DataLoader(train_subset, batch_size=5000, num_workers=cfg.num_workers)

test_subset = Subset(test_dataset, list(range(num_test_samples)))

test_loader = DataLoader(test_subset, batch_size=1000, num_workers=cfg.num_workers)

In [ ]:
trainer = instantiate(cfg.trainer, enable_progress_bar=False, enable_model_summary=False)

## Load models

In [ ]:
from ccmm.utils.utils import load_model_from_artifact

run = wandb.init(project=core_cfg.core.project_name, entity=core_cfg.core.entity, job_type="matching")

# {a: 1, b: 2, c: 3, ..}
symbols_to_seed: Dict[int, str] = {map_model_seed_to_symbol(seed): seed for seed in cfg.model_seeds}

artifact_path = (
    lambda seed: f"{core_cfg.core.entity}/{core_cfg.core.project_name}/{core_cfg.dataset.name}_{core_cfg.model.model_identifier}_{seed}:v0"
)

# {a: model_a, b: model_b, c: model_c, ..}
models: Dict[str, LightningModule] = {
    map_model_seed_to_symbol(seed): load_model_from_artifact(run, artifact_path(seed)) for seed in cfg.model_seeds
}

num_models = len(models)

pylogger.info(f"Using {num_models} models with architecture {core_cfg.model.model_identifier}")

In [ ]:
# always permute the model having larger character order, i.e. c -> b, b -> a and so on ...
symbols = set(symbols_to_seed.keys())
sorted_symbols = sorted(symbols, reverse=False)

# (a, b), (a, c), (b, c), ...
all_combinations = get_all_symbols_combinations(symbols)
# combinations of the form (a, b), (a, c), (b, c), .. and not (b, a), (c, a) etc
canonical_combinations = [(source, target) for (source, target) in all_combinations if source < target]

In [ ]:
pylogger.info(f"Matching the following model pairs: {canonical_combinations}")

In [ ]:
permutation_spec_builder = instantiate(core_cfg.model.permutation_spec_builder)
permutation_spec = permutation_spec_builder.create_permutation_spec()

ref_model = list(models.values())[0]
assert set(permutation_spec.layer_and_axes_to_perm.keys()) == set(ref_model.model.state_dict().keys())

## Git Rebasin 

In [ ]:
from ccmm.matching.matcher import GitRebasinMatcher

matcher = GitRebasinMatcher(name="git_rebasin", permutation_spec=permutation_spec)
pylogger.info(f"Matcher: {matcher.name}")

In [ ]:
from ccmm.matching.utils import get_inverse_permutations

universe_model = models["c"]
map_to_universe = {symb: None for symb in symbols}
map_from_universe = {symb: None for symb in symbols}

for symb, model in models.items():
    permutations, perm_history = matcher(fixed=universe_model.model, permutee=model.model)
    map_to_universe[symb] = permutations
    map_from_universe[symb] = get_inverse_permutations(permutations)

In [ ]:
# A -> B -> C

perm = partial(apply_permutation_to_statedict, permutation_spec)

A_to_univ = perm(map_to_universe["a"], models["a"].model.state_dict())
A_to_B = perm(map_from_universe["b"], A_to_univ)

A_B_C = perm(map_to_universe["b"], A_to_B)
A_B_C = perm(map_from_universe["c"], A_B_C)

ref_model = copy.deepcopy(models["a"])
ref_model.model.load_state_dict(A_B_C)

In [ ]:
trainer.test(ref_model, test_loader)

## PLOT: variance when merging using pairwise maps on different reference basins

In [ ]:
# Data
categories = ["a", "b", "c", "d", "e"]
test_values = [78.55, 65.32, 78.81, 77.72, 78.61]
train_values = [82.05, 67.28, 82.63, 81.36, 82.02]
mean_test = 75.8
mean_train = 79.06
C2M3_train = 82.41
stddev_test = 5
C2M3_test = 78.90
# Bar width
bar_width = 0.35

# Positions of bars on x-axis
r1 = np.arange(len(categories))
r2 = [x + bar_width for x in r1]

# Create the plot
fig, ax = plt.subplots(figsize=(12, 8))

# Bar plot for test values
bars1 = ax.bar(r1, test_values, color=palette["light red"], width=bar_width, edgecolor="grey", label="Pairwise -- test")
bars2 = ax.bar(
    r2, train_values, color=palette["light orange"], width=bar_width, edgecolor="grey", label="Pairwise -- train"
)

# Adding accuracy scores on top of the bars
for i, value in enumerate(test_values):
    ax.text(i, value + 0.5, f"{value}", ha="center", va="bottom")

for i, value in enumerate(train_values):
    ax.text(i + bar_width, value + 0.5, f"{value}", ha="center", va="bottom")

# Adding C^2M^3 bar
ax.bar(
    len(categories) + 1,
    C2M3_test,
    color=palette["dark blue"],
    width=bar_width,
    edgecolor="grey",
    label="$C^2M^3$ -- test",
)
ax.bar(
    len(categories) + 1 + bar_width,
    C2M3_train,
    color=palette["green"],
    width=bar_width,
    edgecolor="grey",
    label="$C^2M^3$ -- train",
)

# Highlight mean and stddev for test
ax.axhline(y=mean_test, color=palette["light red"], linestyle="--", linewidth=1, label="Pairwise mean -- test")
ax.axhline(y=mean_train, color=palette["light orange"], linestyle="--", linewidth=1, label="Pairwise mean -- train")

# Adding accuracy score for C^2M^3
ax.text(len(categories) + 1, C2M3_test + 0.5, f"{C2M3_test}", ha="center", va="bottom")
ax.text(len(categories) + 1 + bar_width, C2M3_train + 0.5, f"{C2M3_train}", ha="center", va="bottom")

# Set y-axis limits
ax.set_ylim(60, 85)

# Add labels
ax.set_ylabel("Accuracy", fontweight="bold")
ax.set_title("Accuracy of the merged model for different reference models", fontweight="bold")
ax.set_xticks(list(r1) + [len(categories) + 1])
ax.set_xticklabels(categories + ["universe"])

# Add legend
ax.legend(ncol=3, loc="lower center", bbox_to_anchor=(0.5, -0.25))

plt.savefig("figures/pairwise_to_reference_barplot.pdf", bbox_inches="tight")
# Show plot
plt.tight_layout()
plt.show()

## Synchronized

In [ ]:
from ccmm.matching.matcher import FrankWolfeSynchronizedMatcher

fw_matcher = FrankWolfeSynchronizedMatcher(
    name="frank_wolfe", permutation_spec=permutation_spec, initialization_method="identity", max_iter=200
)

In [ ]:
permutations, perm_history = fw_matcher(models, sorted_symbols, canonical_combinations)

In [ ]:
from ccmm.matching.utils import unfactor_permutations

models_permuted_pairwise = {
    symbol: {other_symb: None for other_symb in set(symbols).difference(symbol)} for symbol in symbols
}
pairwise_permutations = unfactor_permutations(permutations)

for fixed, permutee in all_combinations:
    ref_model = copy.deepcopy(models["c"])

    permuted_params = apply_permutation_to_statedict(
        permutation_spec, pairwise_permutations[fixed][permutee], models[permutee].model.state_dict()
    )

    ref_model.model.load_state_dict(permuted_params)
    models_permuted_pairwise[fixed][permutee] = ref_model

In [ ]:
A_to_C = models_permuted_pairwise["c"]["a"]

In [ ]:
trainer.test(A_to_C, test_loader)